## Tutorial 1

This notebook accompanies Example 3.3 in the article and demonstrates the main functionality of the code in this repository. It solves the power sum problem
\begin{align*}
	q_1^2 + q_2^2 &= 2X^4 - 2X^3Y + 3X^2Y^2 - 8XY^3 + 5Y^4 = f_2 \\
	q_1^3 + q_2^3 &= -9X^5Y + 18X^4Y^2 - 9X^3Y^3 + 9X^2 Y^4 - 18XY^5 + 9Y^6 = f_3
\end{align*}

Let us start by defining $f_2$ and $f_3$. 

In [135]:
using Pkg; Pkg.activate("SoSupport"); 
using MosekTools; default_solver=Symbol(Mosek); # change to your favourite solver. MOSEK needs a licence. Comment to use the COSMO solver as a default.  
include("src/pof-decomp.jl");
using LinearAlgebra; using DataFrames; const dp = DynamicPolynomials;

@polyvar y x;
f2 = 2x^4 - 2x^3*y + 3x^2*y^2 - 8x*y^3 + 5y^4
f3 = -9x^5*y + 18x^4*y^2 - 9x^3*y^3 + 9x^2*y^4 - 18x*y^5 + 9y^6
display(f2); display(f3);

2x⁴ - 2yx³ + 3y²x² - 8y³x + 5y⁴

-9yx⁵ + 18y²x⁴ - 9y³x³ + 9y⁴x² - 18y⁵x + 9y⁶

  Activating project at `~/Dokumente/git repositories/powers-of-forms/SoSupport`


First, we compute the unique Gram matrix $G$ of $f_2$ with an SDP solver.


In [136]:
s = calc_sos_attributes(f2);
display(s.G); 
println("Monomials: [x², yx, y²]")


3×3 SymMatrix{Float64}:
  2.0  -1.0  -1.0
 -1.0   5.0  -4.0
 -1.0  -4.0   5.0

Monomials: [x², yx, y²]


The following information was computed with semidefinite programming. In particular, $f_2$ does indeed have a unique Gram matrix.  


In [137]:
display(report(s));

Row,Parameter,Value,Explanation
,Symbol,Any,String
1,n,2,No. of variables
2,k,2,degree
3,solver,Mosek,SDP solver
4,solve_time,0.00327497,"SDP solve time for (G, M_E)"
5,digits,7,Threshold for truncating eigenvalues (in digits)
6,uniquely_sos_representable,true,
7,dual_nondegenerate,true,Whether f lies on exposed face of Σ_2k
8,dim_sosupp,2,
9,spectral_gap_G,"(3.0, 2.13163e-14)","(smallest nontruncated, largest truncated) eigval"


The sum-of-squares support $u = (u_1, u_2)$ is given by the image of G. For the sake of exposition, we rescale and round the polynomials such that their x*y coefficient equals -1. 


In [138]:
display(round.(s.sosupp, digits=4)); # sos support, with coefficient vectors normalized to unit length. 

c = abs.(dp.coefficient.(s.sosupp, x*y))
u_normalized = s.sosupp ./ c;
u_normalized = round.(u_normalized, digits = 6); 

display(u_normalized) # sos support, with coefficient vectors normalized s.t. the x*y coefficient equals 1. 


2-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}}:
 0.8165x² - 0.4082yx - 0.4082y²
 -0.7071yx + 0.7071y²

2-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}}:
 2.0x² - yx - y²
 -yx + y²

We now use the decomposition algorithm from Theorem 2.3. Note that the argument `u` for the function `pof_decompose` is optional. If no value is provided, then pof_decompose calls `calc_sos_attributes` to compute the sum-of-squares support. 

In [157]:
dec = pof_decompose(f2, f3; u=u_normalized);

# similarly, renormalize g2 and g3 to fit 
g2 = round.(dec.g_2, digits=6); println("g2 = $g2");
g3 = round.(dec.g_3, digits=6); println("g3 = $g3");
ℓ1 = round.(dec.ℓ[1], digits=6); println("ℓ1 = $ℓ1");
ℓ2 = round.(dec.ℓ[2], digits=6); println("ℓ2 = $ℓ2");


g2 = 4.5*Y[2]^2 + 0.5*Y[1]^2
g3 = 6.75*Y[2]^3 + 2.25*Y[1]^2*Y[2]
ℓ1 = 1.5*Y[2] + 0.5*Y[1]
ℓ2 = 1.5*Y[2] - 0.5*Y[1]


The power sum decomposition is obtained via backsubstitution. This gives the desired solution: 

In [159]:
q1 = round.(dec.q[1], digits=6); println("q1 = $q1"); # the same as ℓ1(dec.Y=>u_normalized)
q2 = round.(dec.q[2], digits=6); println("q2 = $q2"); # the same as ℓ2(dec.Y=>u_normalized)

q1 = x^2 - 2.0*y*x + y^2
q2 = -x^2 - y*x + 2.0*y^2


Note that `pof_decompose` throws an error, if $u_1,\ldots,u_N$ are not cubically independent. As a sanity check, we may use the function `is_cubically_independent`. Aside from a true/false answer, it also computes the minimum singular value of the coefficient matrix of the 3-fold products $u_iu_ju_k$.  

In [33]:
is_cubically_independent(u_normalized)

(true, 3.307649321514332)